# MRIO eval with Python for MSc theses
Overview of how basic MRIO analysis works
The numbers in this tutorial are adapted to EXIOBASE2 (163 products/industries in 48 regions). The reference year is 2007.
## 1. Load data

In [1]:
import numpy as np
import xlrd, xlwt
import scipy.io



In [37]:
Filestring_Matlab_in      = 'C:\\Users\\spauliuk\\FILES\\Work_Archive\\_Data_Derived\\IO_models\\EXIOBASE2_Mon_48R_2017_3_8_ITC.mat' 

MRIO_L  = scipy.io.loadmat(Filestring_Matlab_in)['EB2_L_ITC'] # The Leontief inverse
MRIO_S  = scipy.io.loadmat(Filestring_Matlab_in)['EB2_S_ITC'] # The stressor matrix
MRIO_Y  = scipy.io.loadmat(Filestring_Matlab_in)['EB2_Y']     # The final demand matrix
MRIO_FY = scipy.io.loadmat(Filestring_Matlab_in)['EB2_FinalDemand_Emissions']     # Emissions of final demand sectors
MRIO_Re = scipy.io.loadmat(Filestring_Matlab_in)['EB2_RegionList']    # List of regions
MRIO_Pr = scipy.io.loadmat(Filestring_Matlab_in)['EB2_ProductNames163']    # List of 163 products, aggregated from 200 original products
MRIO_Ex = scipy.io.loadmat(Filestring_Matlab_in)['Extensions_Labels']    # List of 692 extensions
MRIO_Ca = scipy.io.loadmat(Filestring_Matlab_in)['EB2_CharacterisationCategories']    # List of 124 characterisation categories
MRIO_CU = scipy.io.loadmat(Filestring_Matlab_in)['EB2_CharacterisationUnits']    # List of 124 characterisation categories
MRIO_Cf = scipy.io.loadmat(Filestring_Matlab_in)['EB2_CharacterisationFactors']    # Matrix of characterisation factors

scipy.io.loadmat(Filestring_Matlab_in).keys() # Display all variable names in MRIO data file

dict_keys(['Units', 'EB2_CharacterisationUnits', 'EB2_CharacterisationCategories2', 'Extensions_Labels', 'EB2_FinalDemand_Emissions', '__version__', 'EB2_L_ITC', 'EB2_A_ITC', 'EB2_S_ITC', 'EB2_IndustryNames163', '__header__', 'EB2_CharacterisationCategories', '__globals__', 'EB2_ProductNames163', 'EB2_CharacterisationMethods', 'EB2_Y', 'EB2_CharacterisationFactors', 'EB2_RegionList', 'ScriptConfig'])

Let's first have a look at the shape of the arrays:

In [38]:
print(MRIO_Y.shape)
print(MRIO_L.shape)
print(MRIO_S.shape)
print(MRIO_Cf.shape)

(7824, 288)
(7824, 7824)
(692, 7824)
(124, 692)


## 2. The Y vector:
Y is not a vector but a matrix, where for each of the 48 regions the following 6 final demand groups are recorded:
+ Final consumption expenditure by households
+ Final consumption expenditure by non-profit organisations serving households (NPISH)
+ Final consumption expenditure by government
+ Gross fixed capital formation
+ Changes in inventories
+ Changes in valuables

For national footprint calculations, the latter two columns are usually ignored, as they contain goods that do not provide service during the accounting period. We therefore slice the Y vector.

For example, to obtain the final demand of Germany, we specify:

In [33]:
Y_DE = MRIO_Y[:,30:34]
Y_DE.sum() # Total final demand of Germany in MEUR.

2131815.4625037638

This is because Germany is no. 6 in the country list, so there are five other countries with 6 final demand columns each coming before Germany, and occupying columns 0 to 29. The slicing parameter 30:34 selects columns 30, 31, 32, and 33. 

The first column of Y_DE is the column for household final demand. This column contains the final demand of all German households, for all products in from all regions. For example, final demand for product no. 90, motor vehicles, is fullfilled both with domestic production and with imports from other regions:

In [18]:
Y_DE_HH_MotorVehicles = Y_DE[90::163,0]
Y_DE_HH_MotorVehicles.shape
ind = Y_DE_HH_MotorVehicles.argsort() # sort

for m in range(0,48):
    print(MRIO_Re[ind[47-m]], ' ', Y_DE_HH_MotorVehicles[ind[47-m]], 'MEUR')

Germany                41143.7130859 MEUR
France                 2842.31938362 MEUR
Italy                  1960.87597672 MEUR
Austria                1853.33673309 MEUR
Spain                  1806.32841972 MEUR
Hungary                1723.41290664 MEUR
Czech Republic         1721.1626944 MEUR
United States          1545.8108521 MEUR
United Kingdom         1426.4290719 MEUR
Belgium                1288.56005124 MEUR
Poland                 1250.35164902 MEUR
Slovakia               914.730055012 MEUR
Japan                  817.205253288 MEUR
Netherlands            614.778937028 MEUR
Brazil                 567.803261993 MEUR
Mexico                 540.651379438 MEUR
Turkey                 476.816409415 MEUR
South Korea            476.169766556 MEUR
Sweden                 455.40406584 MEUR
Portugal               303.097152362 MEUR
Finland                238.084275915 MEUR
Slovenia               186.400073126 MEUR
South Africa           164.248718803 MEUR
Romania                159.360433979 M

## 3. First MRIO analyses with EXIOBASE2:
1) Obtain entire global emissions and select household footprint

In [34]:
GlobalFootprint_ByCountryAndCategory = MRIO_S.dot(MRIO_L).dot(MRIO_Y)
GlobalFootprint_ByCountryAndCategory.shape

(692, 288)

The result is a 692 emissions by 288 final demand categories array. The global total emissions are obtained by selecting specific rows:

In [35]:
print(str(GlobalFootprint_ByCountryAndCategory[0,:].sum()/1e9) + ' Mt') # global emissions from CO2 combusion to air, Mt

24645.0157601 Mt


The national footprints are obtained by selecting specific columns:

In [36]:
HouseholdFootprint_De = GlobalFootprint_ByCountryAndCategory[:,30]
print(HouseholdFootprint_De[0]/1e9) # CO2 - combustion', 'air', 'Mt'

557.172725027


2) Aggregate emissions to midpoint indicators

Done by multiplying the emissions array with the matrix of characterisation factors.
We choose the global warming potential with 100 year time horizon taken from the CML method.

In [39]:
MRIO_MidPoints = MRIO_Cf.dot(GlobalFootprint_ByCountryAndCategory)
MRIO_MidPoints.shape

(124, 288)

In [41]:
MRIO_MidPoints[5,:].sum()/1e12 # 2007 global carbon emissions in Gt of CO2-eq

32.034988361907921

In [42]:
MRIO_MidPoints[5,30]/1e9 # household carbon footprint, Germany, 2007, in Mt

758.50860278613288

4) Product-specific footprints, example: German household consumption.
Equation: 

$$c = C\cdot S\cdot L\cdot\hat{Y}$$

In [43]:
Y_DE_HH_expanded = np.diag(MRIO_Y[:,30])
Y_DE_HH_expanded.shape

(7824, 7824)

In [44]:
MRIO_MidPoints_DE_HH_Productspecific = MRIO_Cf.dot(MRIO_S).dot(MRIO_L).dot(Y_DE_HH_expanded)
MRIO_MidPoints_DE_HH_Productspecific.shape

The carbon footprint of motorvehicles can be summed up over all countries of origin or listed separately, by country of origin:

In [47]:
print(MRIO_MidPoints_DE_HH_Productspecific[5,90::163].sum()/1e9)
# Carbon footprint of producing all motor vehicles bought by German households, 2007, Mt of CO2-eq

50.3759406993


In [51]:
MidPoints_DE_HH_MotorVehicles = MRIO_MidPoints_DE_HH_Productspecific[5,90::163]
ind = MidPoints_DE_HH_MotorVehicles.argsort() # sort

for m in range(0,48):
    print(MRIO_Re[ind[47-m]], ' ', MidPoints_DE_HH_MotorVehicles[ind[47-m]]/1e9, 'Mt of CO2-eq')

Germany                29.0288438105 Mt of CO2-eq
France                 1.8042043912 Mt of CO2-eq
Czech Republic         1.76819536791 Mt of CO2-eq
Spain                  1.57319259915 Mt of CO2-eq
Hungary                1.52702302841 Mt of CO2-eq
Italy                  1.50177897441 Mt of CO2-eq
Poland                 1.42838070948 Mt of CO2-eq
United States          1.29442944691 Mt of CO2-eq
Austria                1.20636653271 Mt of CO2-eq
United Kingdom         1.09763446395 Mt of CO2-eq
Slovakia               0.956359549655 Mt of CO2-eq
Belgium                0.923748406517 Mt of CO2-eq
Japan                  0.797121361795 Mt of CO2-eq
Turkey                 0.777979863362 Mt of CO2-eq
Brazil                 0.713466100144 Mt of CO2-eq
South Korea            0.591595199529 Mt of CO2-eq
Mexico                 0.456195234944 Mt of CO2-eq
Netherlands            0.429841637284 Mt of CO2-eq
South Africa           0.392841077965 Mt of CO2-eq
China                  0.389759888595 Mt o

5) Sector-specific emissions, example: German household consumption.
Equation: 

$$c = C\cdot S\cdot \widehat{L\cdot Y}$$

In [50]:
MRIO_MidPoints_DE_HH_Sectorspecific = MRIO_Cf.dot(MRIO_S).dot(np.diag(MRIO_L.dot(MRIO_Y[:,30])))
MRIO_MidPoints_DE_HH_Sectorspecific.shape

(124, 7824)

In [67]:
MidPoints_DE_HH_BySector = MRIO_MidPoints_DE_HH_Sectorspecific[5,:]
ind = MidPoints_DE_HH_BySector.argsort() # sort

for m in range(0,50):
    print(MRIO_Re[ind[7823-m] // 163], ', ', MRIO_Pr[ind[7823-m] % 163][0:30], ': ', MidPoints_DE_HH_BySector[ind[7823-m]]/1e9, 'Mt of CO2-eq')

Germany              ,  Electricity by coal            :  83.9551335789 Mt of CO2-eq
Russia               ,  Crude petroleum and services r :  38.3075847924 Mt of CO2-eq
China                ,  Electricity by coal            :  33.9723577741 Mt of CO2-eq
Germany              ,  Chemicals nec                  :  18.743359677 Mt of CO2-eq
Germany              ,  Air transport services         :  17.9608915718 Mt of CO2-eq
China                ,  Basic iron and steel and of fe :  17.3171793249 Mt of CO2-eq
Germany              ,  Steam and hot water supply ser :  16.310152851 Mt of CO2-eq
Germany              ,  Collected and purified water,  :  12.6646130648 Mt of CO2-eq
Germany              ,  Refinery products              :  12.3027854147 Mt of CO2-eq
Germany              ,  Electricity by gas             :  12.2004074161 Mt of CO2-eq
Germany              ,  Other business services        :  11.8845467843 Mt of CO2-eq
RoW Europe           ,  Electricity by coal            :  11.342181